# Single Agent Organ Generator V2

This notebook provides a comprehensive demonstration of the Single Agent Organ Generator V2 workflow, featuring the adaptive Rule Engine for intelligent requirements capture.

## Key Features

| Feature | Description |
|---------|-------------|
| **Adaptive Rule Engine** | Intelligent questioning based on missing information, ambiguous language, and feasibility constraints |
| **Three Rule Families** | Completeness (A), Ambiguity (B), and Conflict (C) rules |
| **Resolution Strategy** | Three-tier approach: Infer from text, propose defaults, ask targeted questions |
| **Question Prioritization** | Questions ranked by rework cost to minimize iteration |
| **Organ-Specific Variants** | Tailored questions for liver, kidney, lung, heart, and generic structures |
| **Multi-Object Support** | Generate multiple structures per project with independent workflows |

## 1. Setup and Imports

## Important: Execution Modes

Understanding the available execution modes is essential for effective use of this workflow.

| Mode | Description | Recommended Use Case |
|------|-------------|---------------------|
| **Prompt-only** (default) | LLM returns text responses without code execution | Planning, prototyping, specification generation |
| **Execution-enabled** | LLM-generated code runs in a sandboxed environment | Testing with controlled output |
| **Spec-only** (recommended) | LLM produces structured specification; Python executes generation locally | Production workflows |

**Note:** By default, `AgentRunner` has `auto_execute_code=False`. The LLM describes intended actions but does not create files. For production use, we recommend the spec-only approach where the LLM produces a structured specification and Python executes generation locally.

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(".."))
from automation import (
    SingleAgentOrganGeneratorV2, WorkflowState, ProjectContext, ObjectContext,
    ObjectRequirements, QUESTION_GROUPS, ORGAN_QUESTION_VARIANTS,
    detect_organ_type, get_tailored_questions, run_single_agent_workflow,
    # Rule Engine components
    RuleEngine, IntentParser, QuestionPlanner,
    RuleFlag, ProposedDefault, RuleEvaluationResult, PlannedQuestion,
    run_rule_based_capture,
)
print("All imports successful!")

## 2. Configuration

Configure the LLM provider and model for the workflow.

In [ ]:
API_KEY = os.environ.get('OPENAI_API_KEY', None)
PROVIDER = 'openai'
MODEL = 'gpt-4'
print(f'Provider: {PROVIDER}, Model: {MODEL}, API Key: {API_KEY is not None}')

## 3. Rule Engine

The workflow employs an adaptive rule engine that intelligently determines which questions to ask. Rather than presenting a fixed sequence of questions, the rule engine evaluates:

1. **Missing Information** (Family A - Completeness): Required fields that must be populated
2. **Ambiguous Language** (Family B - Ambiguity): Terms that require clarification
3. **Feasibility Issues** (Family C - Conflict): Specifications that would fail or produce impractical results

### 3.1 IntentParser

The IntentParser extracts explicit values from user text and detects ambiguities that require clarification.

In [ ]:
# Test IntentParser with different intents
test_intents = [
    'box 2x6x3 cm with diameter 1mm',
    'dense liver-like network with inlet on the left side',
    'symmetric branching with 2 inlets',
    'perfusion network for tissue engineering',
]

print('IntentParser Analysis:')
print('=' * 70)
for intent in test_intents:
    parser = IntentParser(intent)
    print(f"\nIntent: '{intent}'")
    print(f"  Spatial ambiguity: {parser.has_spatial_ambiguity()}")
    print(f"  Vague quantifiers: {parser.has_vague_quantifiers()}")
    print(f"  Implicit I/O: {parser.has_implicit_io()}")
    print(f"  Symmetry ambiguity: {parser.has_symmetry_ambiguity()}")
    if parser.extracted_values:
        print(f"  Extracted values: {parser.extracted_values}")

### 3.2 RuleEngine

The RuleEngine evaluates requirements against all three rule families and determines what information is needed.

In [ ]:
# Create empty requirements and evaluate
requirements = ObjectRequirements()
engine = RuleEngine(organ_type='liver')

result = engine.evaluate(requirements, intent='dense liver network')

print('RuleEngine Evaluation:')
print('=' * 70)
print(f"Generation ready: {result.is_generation_ready}")
print(f"\nMissing fields ({len(result.missing_fields)}):")
for flag in result.missing_fields[:3]:
    print(f"  [{flag.severity}] {flag.message}")
print(f"\nAmbiguity flags ({len(result.ambiguity_flags)}):")
for flag in result.ambiguity_flags:
    print(f"  [{flag.severity}] {flag.message}")
print(f"\nProposed defaults ({len(result.proposed_defaults)}):")
for prop in result.proposed_defaults[:3]:
    print(f"  {prop.field}: {prop.value} ({prop.reason})")

### 3.3 QuestionPlanner

The QuestionPlanner generates the minimal set of questions needed, ranked by rework cost.

In [ ]:
# Plan questions based on evaluation result
planner = QuestionPlanner()
questions = planner.plan(result, max_questions=4)

print('Planned Questions (ranked by rework cost):')
print('=' * 70)
for i, q in enumerate(questions, 1):
    default_str = f' [{q.default_value}]' if q.default_value else ''
    print(f"{i}. [{q.rework_cost.upper()}] {q.question_text}{default_str}")
    print(f"   Field: {q.field}")
    print(f"   Reason: {q.reason}")
    print()

### 3.4 Resolution Strategy Comparison

The rule engine employs a three-tier resolution strategy. More explicit user intents result in fewer questions:

In [ ]:
# Demonstrate how explicit values reduce questions
explicit_intent = 'box 2x6x3 cm, diameter 1mm, 2 inlets, 1 outlet'
vague_intent = 'dense branching network on the left side'

for intent in [explicit_intent, vague_intent]:
    parser = IntentParser(intent)
    req = ObjectRequirements()
    eng = RuleEngine()
    res = eng.evaluate(req, intent)
    qs = planner.plan(res, max_questions=4)
    
    print(f"\nIntent: '{intent}'")
    print(f"  Extracted values: {len(parser.extracted_values)}")
    print(f"  Ambiguities detected: {len(res.ambiguity_flags)}")
    print(f"  Questions needed: {len(qs)}")

## 4. Dynamic Question Variance

The workflow automatically detects the organ type from user descriptions and tailors questions accordingly for Groups C (Inlets/Outlets), D (Topology), and E (Geometry).

### 4.1 Organ Type Detection

The system identifies organ type from keywords in the user's description.

In [ ]:
test_intents = [
    'liver vascular network',
    'kidney with renal arteries',
    'lung pulmonary tree',
    'coronary arteries for heart',
    'generic tubular structure',
]
print('Organ Detection Results:')
print('-' * 50)
for intent in test_intents:
    organ = detect_organ_type(intent)
    print(f"{organ:8} <- {intent}")

### 4.2 Tailored Questions by Organ Type

Each organ type receives customized questions appropriate to its anatomical characteristics.

In [ ]:
print('Group C (Inlets/Outlets) - First Question by Organ Type:')
print('=' * 60)
for organ in ['liver', 'kidney', 'lung', 'heart', 'generic']:
    q = get_tailored_questions(f'{organ} network')
    group_c = q['C']
    first_q = group_c['questions'][0]
    print(f"\n{organ.upper()} ({group_c['name']}):\n  Q: {first_q[1]}\n  Default: {first_q[2]}")

## 5. Workflow States

The workflow progresses through a defined sequence of states, from project initialization to final artifact export.

In [ ]:
print('Workflow States:')
for i, state in enumerate(WorkflowState, 1):
    print(f"{i:2}. {state.value}")

## 6. Question Groups (Reference)

The following question groups are maintained for reference and backward compatibility. The rule engine (Section 3) is the recommended approach for requirements capture, as it asks only what is needed based on the current state of the specification.

In [ ]:
print('Question Groups:')
for key, group in QUESTION_GROUPS.items():
    print(f"Group {key}: {group['name']} ({len(group['questions'])} questions)")

## 7. Organ-Specific Question Variants

These variants provide organ-specific phrasing for questions. The rule engine automatically detects organ type from user intent and applies appropriate variants to make the conversation more natural.

In [ ]:
print('Organ-Specific Variants:')
for organ, variants in ORGAN_QUESTION_VARIANTS.items():
    print(f"\n{organ.upper()}: {variants['description']}")
    for g in ["C", "D", "E"]:
        if g in variants:
            print(f"  Group {g}: {variants[g]['name']}")

## 8. Running the Workflow

Launch the interactive workflow to generate organ structures.

In [ ]:
# Quick start (uncomment to run):
# context = run_single_agent_workflow(provider=PROVIDER, model=MODEL, api_key=API_KEY)
print('Uncomment above to run interactive workflow')

## 9. Per-Object Folder Structure

Each object in a project maintains a complete artifact history in a structured folder hierarchy.

In [ ]:
print("""outputs/<project>/
├── project.json
└── objects/<object>/
    ├── 00_intent/     (intent.txt, requirements_v001.json)
    ├── 01_spec/       (spec_v001.json)
    ├── 02_code/       (generate_v001.py)
    ├── 03_outputs/    (network_v001.json)
    ├── 04_mesh/       (mesh_v001_network.stl)
    ├── 05_analysis/   (analysis_v001.json)
    ├── 06_validation/ (validation_v001.json)
    ├── 07_iterations/
    └── 08_final/      (void.stl, manifest.json)""")

## 10. Requirements Schema

The requirements schema comprises nine sections that fully specify an organ structure.

In [ ]:
sections = [
    ("1. Identity", "object_name, slug, version"),
    ("2. Frame of Reference", "origin, axes, viewpoint, units"),
    ("3. Domain", "type, size, center, margin"),
    ("4. Inlets/Outlets", "positions, radii, directions"),
    ("5. Topology Intent", "style, target_terminals, max_depth"),
    ("6. Geometry Intent", "segment_length, tortuosity, branch_angle"),
    ("7. Constraints", "min_radius, min_clearance, boundary_buffer"),
    ("8. Embedding & Export", "domain, voxel_pitch, stl_units"),
    ("9. Acceptance Criteria", "min_radius, terminals_range, watertight"),
]
print('Requirements Schema:')
for section, fields in sections:
    print(f"{section}: {fields}")

## Summary

This notebook demonstrated the following capabilities:

| Topic | Description |
|-------|-------------|
| **Rule Engine** | Adaptive requirements capture with three rule families |
| **Dynamic Variance** | Organ-specific question tailoring for liver, kidney, lung, heart |
| **Workflow States** | Ten-step workflow from initialization to completion |
| **Question Groups** | Seven groups (A-G) covering all specification aspects |
| **Requirements Schema** | Nine-section schema for complete structure specification |
| **Folder Structure** | Per-object artifact organization with versioning |